In [1]:


import os
import json
import boto3
import sklearn
import socket
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


print('user:', os.environ['JUPYTERHUB_SERVICE_PREFIX'])

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return "{}proxy/{}/jobs/".format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

# small fix to enable UI views
SparkContext.uiWebUrl = property(uiWebUrl)

# spark configurtion in local regime 
conf = SparkConf().set('spark.master', 'local[*]').set('spark.driver.memory', '8g')

#some needed objects
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

user: /user/st084310/


# Homework 1

Transform text file "The Project Gutenberg eBook of Frankenstein, by Mary Wollstonecraft (Godwin) Shelle" into TF-IDF. Take row as "document".

### Part 1: 
- read text file as dataframe 
- filter out non-letters and empty strings 
- transform into dataframe doc_id -> tf_idf vector 


### Part 2:
- read text file as RDD
- filter out non-letters and empty strings 
- transform into rdd in format doc_id -> tf_idf vector


### Org part: 
I'm waiting your HW's as self-sufficient jupyter notebooks in github repository. 

Please, fill this table in specified comment with:

your name / github link / telegram (optionally, if u want to discuss some) / 

Fill the comment please and i will add your data in a few days

https://docs.google.com/spreadsheets/d/1p3yLsXqX2dp_TrPwNcikcS5FP_PTM0_gnSOzGn5Gn1E/edit#gid=0

Feel free to text me if u have some questions 

### Deadline: 05.05.2021 included

Dear students, dead in "deadline" means *dead*. This deadline is not for you - it's for me. Deadlines informs me from which point i should start to score your HWs.  You can commit anything after deadline but it's not guaranteed that I'll take it into account. It's possible to move deadline only for the whole group not "just for me plz cause I was ill / detentioned / skipped this message". 

### NB(!) 

It's not allowed to use TF-IDF code from Spark internal libraries. 
It's not allowed to cast DF/RDD into pandas and use scikit-learn. Please, keep it spark. 


## Part 1

Reading text file as rdd

In [2]:


result_prefix = "malyutin_demo_hw1"

filepath = "file:///home/jovyan/shared/lectures_folder/84-0.txt"
from pyspark.sql.functions import monotonically_increasing_id

dataframe = sc.textFile(f"{filepath}")\
    .map(lambda x: (x,))\
    .toDF()\
    .select(F.col("_1").alias("text"))\
    .withColumn("id", monotonically_increasing_id())

dataframe.show()

+--------------------+---+
|                text| id|
+--------------------+---+
|The Project Guten...|  0|
|                    |  1|
|This eBook is for...|  2|
|most other parts ...|  3|
|whatsoever. You m...|  4|
|of the Project Gu...|  5|
|www.gutenberg.org...|  6|
|will have to chec...|  7|
|   using this eBook.|  8|
|                    |  9|
| Title: Frankenstein| 10|
|       or, The Mo...| 11|
|                    | 12|
|Author: Mary Woll...| 13|
|                    | 14|
|Release Date: 31,...| 15|
|[Most recently up...| 16|
|                    | 17|
|   Language: English| 18|
|                    | 19|
+--------------------+---+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import string
import re

def process_string(data):

    punct_removed = re.sub(r'[^\w\s]','',data)
    words = punct_removed.lower().split(" ")
    
    
    return list(filter(lambda x: len(x) > 0, words))


process_string_udf = udf(lambda z: process_string(z), ArrayType(StringType()))

In [4]:


by_words = dataframe\
    .select(process_string_udf(F.col("text")).alias("by_words"))\
    .where(F.size(F.col("by_words")) > 1)


by_words.show()

+--------------------+
|            by_words|
+--------------------+
|[the, project, gu...|
|[this, ebook, is,...|
|[most, other, par...|
|[whatsoever, you,...|
|[of, the, project...|
|[wwwgutenbergorg,...|
|[will, have, to, ...|
|[using, this, ebook]|
|[title, frankenst...|
|[or, the, modern,...|
|[author, mary, wo...|
|[release, date, 3...|
|[most, recently, ...|
| [language, english]|
|[character, set, ...|
|[produced, by, ju...|
|[further, correct...|
|[start, of, the, ...|
|[or, the, modern,...|
|[by, mary, wollst...|
+--------------------+
only showing top 20 rows



In [5]:
by_words_count = by_words.select(F.explode(F.col("by_words")).alias("word"))\
    .groupBy(F.col("word")).count()\
    .orderBy(F.col("count").desc())

by_words_count.show()

+-----+-----+
| word|count|
+-----+-----+
|  the| 4377|
|  and| 3038|
|    i| 2840|
|   of| 2762|
|   to| 2169|
|   my| 1773|
|    a| 1441|
|   in| 1187|
| that| 1029|
|  was| 1022|
|   me|  861|
| with|  713|
|  but|  689|
|  had|  686|
|  you|  644|
|   he|  604|
|which|  565|
|   it|  561|
|   as|  536|
|  not|  536|
+-----+-----+
only showing top 20 rows



In [6]:
by_words_count.repartition(1)\
    .write.mode("overwrite").csv(f"df_by_words.txt")

## Part 2

Reading text file into RDD

In [9]:
result_prefix = "malyutin_demo_hw1"

filepath = "file:///home/jovyan/shared/lectures_folder/84-0.txt"

In [66]:
rddText = sc.textFile(f"{filepath}").repartition(1)
rddText.take(10)

['The Project Gutenberg eBook of Frankenstein, by Mary Wollstonecraft (Godwin) Shelley',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org. If you are not located in the United States, you',
 'will have to check the laws of the country where you are located before',
 'using this eBook.',
 '']

In [67]:
wordsRDD = rddText.flatMap(lambda line: line.lower().split(" ")).filter(lambda line: len(line) > 1)
wordCountRDD= wordsRDD.map(lambda word: (word, 1))

resultRDD = wordCountRDD.reduceByKey(lambda a, b: a + b)


In [68]:
resultRDD.take(10)

[('the', 4340),
 ('project', 83),
 ('gutenberg', 25),
 ('ebook', 8),
 ('of', 2759),
 ('frankenstein,', 8),
 ('by', 482),
 ('mary', 3),
 ('wollstonecraft', 3),
 ('(godwin)', 3)]

In [69]:

# write it back to HDFS
resultRDD.saveAsTextFile(f"rdd_6")